# AXI I2S Transmitter
- Upload `Overlay/antminerS9I2S` into Antminer S9 via SFTP into folder `/home/xilinx/pynq/overlays/`
- Tutorial Creating AXI I2S Overlay : https://github.com/Kampi/ZYBO/tree/master/projects/DigitalAudio<br><br>
<img src="resource/I2S DAC Wiring.jpg" width="700px">

In [41]:
from pynq import Overlay, MMIO
import numpy as np
import time

In [42]:
overlay = Overlay('antminerS9I2S.bit')

In [ ]:
overlay?

In [28]:
fifo = MMIO(overlay.ip_dict['FIFO']['phys_addr'], 0x10000)
clk_wizard = MMIO(overlay.ip_dict['ClockingWizard']['phys_addr'], 0x10000)

In [29]:
# AudioPlayer Configuration
class AudioPlayer:
    def __init__(self, fifo, clk_wizard):
        self.fifo = fifo
        self.clk_wizard = clk_wizard
        self.test_signal_cycles = 0
        self.use_test_signal = False
        self.busy = False
        self.fifo_buffer_size = 256  # Adjust based on design

    def send_test_signal(self, cycles):
        """Send a test sine wave signal."""
        self.test_signal_cycles = cycles
        self.use_test_signal = True
        self.busy = True

        test_signal = self.generate_test_signal()
        self.write_fifo(test_signal)
        print("[INFO] Test signal sent!")

    def write_fifo(self, data):
        """Write data to FIFO."""
        for word in data:
            self.fifo.write(0x00, int(word))  # Assuming 0x0 is FIFO data register
        print("[INFO] FIFO write complete!")

    def generate_test_signal(self):
        """Generate a sine wave as a test signal."""
        t = np.linspace(0, 2 * np.pi, self.fifo_buffer_size)
        sine_wave = (np.sin(t) * 0x7FFF).astype(np.int16)
        return [(sample << 16) | sample for sample in sine_wave]  # Stereo signal

    def change_freq(self, sample_rate):
        """Configure the Clocking Wizard for a specific sample rate."""
        if sample_rate == 44100:
            clk_settings = {"DIVCLK_DIVIDE": 5, "CLKFBOUT_MULT": 42, "DIVIDE": 93}
        elif sample_rate == 48000:
            clk_settings = {"DIVCLK_DIVIDE": 3, "CLKFBOUT_MULT": 23, "DIVIDE": 78}
        elif sample_rate == 96000:
            clk_settings = {"DIVCLK_DIVIDE": 3, "CLKFBOUT_MULT": 23, "DIVIDE": 39}
        else:
            raise ValueError("Unsupported sample rate!")

        # Update Clocking Wizard registers (hypothetical example)
        self.clk_wizard.write(0x0, clk_settings["DIVCLK_DIVIDE"])  # Register offsets are placeholders
        self.clk_wizard.write(0x4, clk_settings["CLKFBOUT_MULT"])
        self.clk_wizard.write(0x8, clk_settings["DIVIDE"])

        print(f"[INFO] Clocking Wizard updated for {sample_rate} Hz.")

    def is_busy(self):
        """Check if the player is busy."""
        return self.busy


In [30]:
# Initialize AudioPlayer
player = AudioPlayer(fifo, clk_wizard)

In [ ]:
time.sleep(3)

#player.change_freq(48000)  # Set sample rate
for __ in range(10):
    player.send_test_signal(40000)  # Send test signal
# while player.is_busy():
#     pass  # Wait for the operation to complete

_________

In [57]:
from pynq import Overlay, MMIO
import sys
import math
import numpy as np
import time

# Load the bitstream
overlay = Overlay('antminerS9I2S.bit')

# Access the AXI FIFO MM IP blocks
fifo_i2s = overlay.axi_fifo_mm_s_1

# Define constants
PERIODSAMPLES = 128
AMP = 16384

class ADAU1761Config:
    def __init__(self, fifo_i2s):
        self.fifo_i2s = fifo_i2s
        self.chip_addr = 0
        self.word_size = 4

def adau1761_init(codec):
    # Reset the FIFO
    print("Resetting FIFO...")
    fifo_i2s.register_map.TDFR = 0xA5A5  # Reset FIFO
    time.sleep(0.01)  # Allow reset to complete

    # Clear FIFO status
    fifo_i2s.register_map.TDFR = 0x0000  # Clear reset
    fifo_i2s.register_map.IER = 0xFFFFFFFF  # Enable interrupts

    # Check FIFO status
    status = int(fifo_i2s.register_map.TDFV)  # Force conversion to integer
    print(f"FIFO status: 0x{status:X}")
    if status == 0:
        print("FIFO not empty. Initialization failed.")
        return -5

    print("FIFO initialized successfully.")
    return 0

def initialize_fifo():
    try:
        # Perform any necessary resets or status checks
        fifo_i2s.register_map.SRR = 0xA5  # Reset FIFO
        print("FIFO reset command sent.")

        # Clear any pending interrupts
        fifo_i2s.register_map.TDFR = 0x1  # Transmit Reset
        fifo_i2s.register_map.RDFR = 0x1  # Receive Reset
        fifo_i2s.register_map.IER = 0x0  # Disable all interrupts
        print("FIFO interrupts cleared.")

        # Check FIFO status
        status = fifo_i2s.register_map.TDFV  # Transmit Data FIFO Vacancy
        print(f"FIFO status after initialization: 0x{status:X}")

        return 0  # Success

    except Exception as e:
        print(f"FIFO initialization failed: {e}")
        return -4  # Error code


def adau1761_i2s_write(codec, left, right):

    # Wait until there is space in the transmit FIFO
    dot_count = 0
    while int(fifo_i2s.register_map.TDFR) != 0:
        dots = '.' * dot_count
        sys.stdout.write(f"I2S FIFO full. Waiting{dots:<3}\r")  # Pad with spaces to clear extra dots
        sys.stdout.flush()

        dot_count = (dot_count + 1) % 4
        time.sleep(0.5)  # Sleep to avoid busy-waiting
    
    # Combine left and right data and send to FIFO
    word = (left << 16) | right
    fifo_i2s.write(0x00, int(word))
    fifo_i2s.register_map.TDFR = 2  # Set the length of the word


In [58]:
try :
    # Initialize codec
    codec = ADAU1761Config(fifo_i2s)
    print("Initializing ADAU1761 ...")
    status = adau1761_init(codec)
    if status == 0:
        print("OK\n\n")
    else:
        print("FAILED\n\n")

    # Prepare sine wave data for left and right channels
    left = np.zeros(PERIODSAMPLES, dtype=np.int16)
    right = np.zeros(PERIODSAMPLES, dtype=np.int16)

    for i in range(PERIODSAMPLES):
        left[i] = int(np.cos(i / PERIODSAMPLES * 2 * np.pi) * AMP)
        right[i] = int(np.sin(i / PERIODSAMPLES * 2 * np.pi) * AMP)

    # Loop to send audio data continuously
    print("Sending audio data...")
    while True:
        for i in range(PERIODSAMPLES):
            adau1761_i2s_write(codec, left[i], right[i])

except KeyboardInterrupt:
    print("\n\nGoodbye")

Initializing ADAU1761 ...
Resetting FIFO...
FIFO status: 0x1FC
FIFO initialized successfully.
OK


Sending audio data...


AttributeError: 'RegisterTDFV' object has no attribute 'read'